In [1]:
!pip install wandb

     |████████████████████████████████| 1.4MB 6.5MB/s 
     |████████████████████████████████| 102kB 9.3MB/s 
     |████████████████████████████████| 112kB 36.2MB/s 
     |████████████████████████████████| 102kB 9.0MB/s 
     |████████████████████████████████| 163kB 40.0MB/s 
     |████████████████████████████████| 71kB 3.5MB/s 
     |████████████████████████████████| 71kB 6.5MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp36-none-any.whl size=6489 sha256=3ef611677aabe5b54d457265573340eea3c00ad7628e7b4f09deaf1cd0f121de
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for watchdog: filename=watchdog-0.10.3-cp36-none-any.whl size=73870 sha256=fdfc83c67cb0843146162980c3e78334489285fddcfc7615ea3e0c60962d4a57
  Stored in directory: /root/.cache/pip/wheels/a8/1d/38/2c19bb311f67cc7b4d07a2ec5ea36ab1a0a0ea50db994a5bc7
  Created wheel for gql: filename=gql-0.2.0-cp36-none-any.whl size=7630 sha256=5055

In [2]:
!wandb login

wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: 2
wandb: You chose 'Use an existing W&B account'
wandb: You can find your API key in your browser here: https://app.wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: You can find your API key in your browser here: https://app.wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: dff003aa03e7d25df35a840b6f0660ae9675efb4
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
Successfully logged in to Weights & Biases!


In [3]:
#launch in colab
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [4]:
#GPU INFO
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Fri Jul 24 20:17:05 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.51.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
import sys
#launch in colab
sys.path.append('/content/drive/My Drive/Colab_Notebooks/git/PFM_Noisy_Labels/FOOD101')


In [6]:
# Internal files
import config
import dataloader
import models

# Pytorch libraries
import torch
import torch.backends.cudnn as cudnn

import os
import time

import wandb


noise file /content/drive/My Drive/Colab_Notebooks/git/PFM_Noisy_Labels/FOOD101/data/noisy_label_kv70_sim.txt generated with noise: 0.7



In [7]:
import numpy as np
import random

random.seed(0)
np.random.seed(0)

import pandas as pd
if config.noise_validation == True:
        df_labels = pd.read_csv(config.drive_dir+"/data/" + "clean_label_kv.txt", header = None, sep = " ")
        df_labels.columns = ["dir", "clean_label"]
    
        df_val = pd.read_csv(config.drive_dir+"/data/clean_val_key_list.txt", header = None, sep = " ")
        df_val.columns = ["dir"]
    
        df_val = df_val.merge(df_labels, how = 'left', left_on = "dir", right_on = "dir")


        #generate a list of random labels
        randomlist = []
        set_classes = set(df_val.clean_label)
        for i in range(0,df_val.shape[0]):
            n = random.randint(min(set_classes),max(set_classes))
            randomlist.append(n)
        df_val["random_class"] = randomlist
    
        #column of probabilities (uniform)
        df_val["p"] = np.random.uniform(0,1,df_val.shape[0])
    
        #use the random list to generate noise in effective percentage r
        N_CLASSES = len(set_classes)
        NOISE_PERCENTAGE = config.r*N_CLASSES/(N_CLASSES-1)
        noisy_class = []
        for index, row in df_val.iterrows():
            if row["p"]<= NOISE_PERCENTAGE:
                noisy_class.append( row["random_class"])
            else:
                noisy_class.append(row["clean_label"])
    
        df_val["noisy_class"] = noisy_class
    
        df_labels["noisy_class"] = df_labels["clean_label"]
        df_val_relabeled = df_val[["dir","noisy_class"]].copy()


        #read noisy labels
        df_labels = pd.read_csv(config.drive_dir+"/data/" + config.train_labels_file, header = None, sep = " ")
        df_labels.columns = ["dir", "noisy_class"]
        df_val = pd.read_csv(config.drive_dir+"/data/clean_val_key_list.txt", header = None, sep = " ")
        df_val.columns = ["dir"]
        val_list = list(df_val.dir)
        def is_validation(x):
          if (x in val_list):
            return(1)
          return(0)

        #separate noisy train and clean test from validation
        df_test = pd.read_csv(config.drive_dir+"/data/clean_test_key_list.txt", header = None, sep = " ")
        df_test.columns = ["dir"]
        test_list = list(df_test.dir)
        test_list = ['/images/' + x + '.jpg' for x in test_list]
        def is_test(x):
          if (x in test_list):
            return(1)
          return(0)
        df_labels.drop_duplicates(subset='dir', keep="last",  inplace = True)
        df_labels["validation_set"] = [is_validation(x) for x in df_labels.dir]
        df_labels["test_set"] =       [is_test(x) for x in df_labels.dir]
        df_labels["train_set"] = (df_labels["test_set"] + df_labels["validation_set"])*(-1) + 1


        train = df_labels[(df_labels.train_set == 1)  ].copy()
        test  = df_labels[(df_labels.test_set == 1)  ].copy()
        #get unique train
        #get unique test

        #Join noisy validation with previous labels
        df_full = df_val_relabeled.append(test[["dir","noisy_class"]]).append(train[["dir","noisy_class"]])
    
        #export
        filename = config.drive_dir + '/data/' + 'noisy_label_kv'+str(int(config.r*100))+'_sim.txt'
        df_full["noisy_class"] = [int(x) for x in df_full["noisy_class"]]
        df_full[["dir", "noisy_class"]].to_csv(filename, sep=' ', index=False,header = False)
        print("\nnoise file " + filename + " generated with noise: " + str(config.r)+"\n")
        # return(df_train[["directory", "noisy_class"]])
       
        test_validation_labels_file  = 'noisy_label_kv'+str(int(config.r*100))+'_sim.txt'

        df_full


noise file /content/drive/My Drive/Colab_Notebooks/git/PFM_Noisy_Labels/FOOD101/data/noisy_label_kv70_sim.txt generated with noise: 0.7



In [8]:
def get_model():

    # Get model from config
    if config.model == "resnet18":
        model = models.resnet18(pretrained=config.pretrained)
    elif config.model == "resnet34":
        model = models.resnet34(pretrained=config.pretrained)
    elif config.model == 'resnet50':
        model = models.resnet50(pretrained=config.pretrained)
    elif config.model == "resnet101":
        model = models.resnet101(pretrained=config.pretrained)
    elif config.model == "resnet152":
        model = models.resnet152(pretrained=config.pretrained)
    elif config.model == "resnext50_32x4d":
        model = models.resnet34(pretrained=config.pretrained)
    elif config.model == 'resnext101_32x8d':
        model = models.resnet50(pretrained=config.pretrained)
    elif config.model == "wide_resnet50_2":
        model = models.resnet101(pretrained=config.pretrained)
    elif config.model == "wide_resnet101_2":
        model = models.resnet152(pretrained=config.pretrained)
    else:
        raise ValueError('%s not supported'.format(config.model))

    # Initialize fc layer
    (in_features, out_features) = model.fc.in_features, model.fc.out_features
    model.fc = torch.nn.Linear(in_features, out_features)
    return model


def scheduler(epoch):
    global lr
    lr = config.lr
    if epoch > config.start_epoch:
        lr = lr / 10.0
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr


def save_checkpoint(state, filename='checkpoint.pth.tar'):
    torch.save(state, filename)
    if config.use_wandb == True:
        wandb.save(filename)

def save_weights(epoch):
        print('| Saving Weights ...', end="\r")
        save_point = config.drive_dir + '/checkpoint/' + config.checkpoint + '_' + str(epoch) + '.pth.tar'
        save_checkpoint({'state_dict': net.state_dict(), }, save_point)

In [30]:
# Training
def train(epoch):
  net.train()
  train_loss = 0
  correct = 0
  total = 0
  scheduler(epoch)

  # print("train before dataloader")
  for step, (inputs, targets, path) in enumerate(train_loader):
      init_time = time.time()
      if use_cuda:  # GPU settings
          (inputs, targets) = inputs.cuda(), targets.cuda()
      optimizer.zero_grad()
      outputs = net(inputs)
      loss = criterion(outputs, targets)
      loss.backward()
      optimizer.step()

      train_loss += loss.data.item()
      _, predicted = torch.max(outputs.data, 1)
      total += targets.size(0)
      correct += predicted.eq(targets.data).cpu().sum()

      # Grab training results
      # print("| Epoch: {}/{}, step: {}/{}, loss: {:.3f}, acc: {:.3f}, time: {:.3f}".format(epoch,config.num_epochs,step + 1,len(train_loader.dataset),loss.data.item(),100. * correct / total,time.time() - init_time))
      # print("| Epoch: {}/{}, step: {}/{}, loss: {:.3f}, acc: {:.3f}, time: {:.3f}".format(epoch,config.num_epochs,step + 1,len(train_loader.dataset),loss.data.item(),100. * correct / total,time.time() - init_time),end="\r")

      sys.stdout.write('\r')
      sys.stdout.write('| Epoch [%3d/%3d] Iter[%3d/%3d]\t\tLoss: %.4f Acc@1: %.3f%%, time: %.3f'
              %(epoch, config.num_epochs, step+1, (len(train_loader.dataset)//config.batch_size)+1, loss.data.item(), 100.*correct/total,time.time() - init_time))
      sys.stdout.flush()

def valid(epoch):
    global best_acc
    net.eval()
    # valid_loss = 0
    correct = 0
    total = 0
    for step, (inputs, targets, path) in enumerate(valid_loader):
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        with torch.no_grad():
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            # valid_loss += loss.data.item()
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += predicted.eq(targets.data).cpu().sum()

    # Grab validation results
    valid_acc = 100. * correct / total
    valid_results = ("| Epoch: {}/{}, val_loss: {:.3f}, val_acc: {:.3f}, ""lr: {:.6f}".format(epoch,config.num_epochs,loss.data.item(),valid_acc,lr))
    record.write(valid_results + '\n')
    record.flush()

    if config.use_wandb == True:
        wandb.log({'epoch': epoch, 'accy_val' : valid_acc })

    # Save checkpoint when best model
    if valid_acc > best_acc:
        best_acc = valid_acc
        print('| Saving Best Model ...', end="\r")
        save_point = config.drive_dir + '/checkpoint/' + str(config.checkpoint) + '.pth.tar'
        save_checkpoint({'state_dict': net.state_dict(), }, save_point)



def test():
    test_net.eval()
    # test_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets, path) in enumerate(test_loader):
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        with torch.no_grad():
            outputs = test_net(inputs)
            loss = criterion(outputs, targets)

        # test_loss += loss.data.item()
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        # correct += predicted.eq(targets.data).cpu().sum()
        correct += predicted.eq(targets.data).sum()

    # Grab validation results
    test_acc = 100. * correct/total
    test_results = "| test_loss: {:.3f}, test_acc: {:.3f}".format(
        loss.data.item(), test_acc)
    record.write(test_results)
    record.flush()

    print(test_results)
    wandb.log({'test_acc' : test_acc })

In [10]:
record = open(config.drive_dir + '/checkpoint/' + config.checkpoint + '_test.txt', 'w')
# record.write('noise_rate=%s\n' % config.noise_rate)
record.flush()



In [11]:
# Get the original_dataset
loader = dataloader.KeyDataLoader()
train_loader, valid_loader, test_loader = loader.run()

# Hyper Parameter settings
torch.manual_seed(config.seed)
torch.cuda.manual_seed_all(config.seed)
use_cuda = torch.cuda.is_available()

# Networks setup
print('\nModel setup')
print('| Building network: {}'.format(config.model))
net = get_model()
test_net = get_model()

if use_cuda:
    net.cuda()
    test_net.cuda()
    cudnn.benchmark = True

# Instantiate a loss function.
criterion = torch.nn.CrossEntropyLoss()

# Instantiate an optimizer to train the model.
optimizer = torch.optim.SGD(
    net.parameters(), lr=config.lr, momentum=0.9, weight_decay=0.001)

print('\nTraining model')
print('| Training Epochs = ' + str(config.num_epochs))
print('| Initial Learning Rate = ' + str(config.lr))
print('| Optimizer = ' + str(config.optimizer_type))



Model setup
| Building network: resnet50


/content/drive/My Drive/Colab_Notebooks/git/PFM_Noisy_Labels/FOOD101/models/resnet.py:116: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight, mode='fan_out')
/content/drive/My Drive/Colab_Notebooks/git/PFM_Noisy_Labels/FOOD101/models/resnet.py:118: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  nn.init.constant(m.weight, 1)
/content/drive/My Drive/Colab_Notebooks/git/PFM_Noisy_Labels/FOOD101/models/resnet.py:119: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  nn.init.constant(m.bias, 0)
Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/checkpoints/resnet50-19c8e357.pth




Training model
| Training Epochs = 5
| Initial Learning Rate = 0.008
| Optimizer = SGD


In [12]:
# config.num_epochs = 30

In [13]:
#Train
best_acc = 0
for epoch in range(1, 1 + config.num_epochs):
    train(epoch)  
    valid(epoch)
    save_weights(epoch)

In [14]:
print('\nTesting model')

checkpoint = torch.load(config.drive_dir + '/checkpoint/' + config.checkpoint + '.pth.tar')
test_net.load_state_dict(checkpoint['state_dict'])
test()


Testing model
| test_loss: 2.657, test_acc: 63.695


In [15]:
config.checkpoint

'cross_entropy_sim_noise_70_noise_val'

In [31]:
print('\nTesting model')

checkpoint = torch.load(config.drive_dir + '/checkpoint/' + config.checkpoint + '_1.pth.tar')
test_net.load_state_dict(checkpoint['state_dict'])
test()


Testing model


wandb: Wandb version 0.9.4 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


| test_loss: 2.946, test_acc: 47.026


In [32]:
print('\nTesting model')

checkpoint = torch.load(config.drive_dir + '/checkpoint/' + config.checkpoint + '_2.pth.tar')
test_net.load_state_dict(checkpoint['state_dict'])
test()


Testing model


wandb: Wandb version 0.9.4 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


| test_loss: 2.407, test_acc: 63.137


In [33]:
print('\nTesting model')

checkpoint = torch.load(config.drive_dir + '/checkpoint/' + config.checkpoint + '_3.pth.tar')
test_net.load_state_dict(checkpoint['state_dict'])
test()


Testing model


wandb: Wandb version 0.9.4 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


| test_loss: 2.657, test_acc: 63.695


In [34]:
print('\nTesting model')

checkpoint = torch.load(config.drive_dir + '/checkpoint/' + config.checkpoint + '_4.pth.tar')
test_net.load_state_dict(checkpoint['state_dict'])
test()


Testing model


wandb: Wandb version 0.9.4 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


| test_loss: 2.546, test_acc: 62.364


In [35]:
print('\nTesting model')

checkpoint = torch.load(config.drive_dir + '/checkpoint/' + config.checkpoint + '_5.pth.tar')
test_net.load_state_dict(checkpoint['state_dict'])
test()


Testing model


wandb: Wandb version 0.9.4 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


| test_loss: 2.505, test_acc: 59.655


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
%matplotlib inline

im = np.array(Image.open('/content/drive/My Drive/Colab_Notebooks/git/PFM_Noisy_Labels/FOOD101/data/images/breakfast_burrito/3100854.jpg'))
plt.imshow(im)
plt.show()

In [16]:
def TEST():
    test_net.eval()
    # test_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets, path) in enumerate(test_loader):
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        with torch.no_grad():
            outputs = test_net(inputs)
            loss = criterion(outputs, targets)

        # test_loss += loss.data.item()
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += predicted.eq(targets.data).sum()

    # Grab validation results
    test_acc = 100. * correct/total
    test_results = "| test_loss: {:.3f}, test_acc: {:.3f}".format(
        loss.data.item(), test_acc)
    record.write(test_results)
    record.flush()

    print(test_results)
    wandb.log({'test_acc' : test_acc })

In [17]:
TEST()

| test_loss: 2.657, test_acc: 63.695


In [18]:
import pandas as pd
clean = pd.read_csv(config.data_dir + "clean_label_kv.txt",header = None, sep = " ")
clean.columns = ["dir", "noisy_label"]
clean

,dir,noisy_label
0,/images/apple_pie/1005649.jpg,3
1,/images/apple_pie/1014775.jpg,3
2,/images/apple_pie/1026328.jpg,3
3,/images/apple_pie/1028787.jpg,3
4,/images/apple_pie/1043283.jpg,3
...,...,...
100995,/images/waffles/942009.jpg,85
100996,/images/waffles/954028.jpg,85
100997,/images/waffles/96181.jpg,85
100998,/images/waffles/97015.jpg,85


In [19]:
import pandas as pd
noise_generated = pd.read_csv(config.data_dir + config.train_labels_file,header = None, sep = " ")
noise_generated.columns = ["dir", "clean_label"]

In [20]:
import pandas as pd
dir = pd.read_csv(config.data_dir + config.train_dir, header = None)
dir.columns = ["dir"]

In [21]:
noise_generated

,dir,clean_label
0,/images/apple_pie/1005649.jpg,49
1,/images/apple_pie/1014775.jpg,3
2,/images/apple_pie/1026328.jpg,53
3,/images/apple_pie/1028787.jpg,5
4,/images/apple_pie/1043283.jpg,33
...,...,...
75745,/images/waffles/981485.jpg,1
75746,/images/waffles/98238.jpg,30
75747,/images/waffles/982668.jpg,91
75748,/images/waffles/995085.jpg,80


In [22]:
dir

,dir
0,/images/apple_pie/1350394.jpg
1,/images/apple_pie/1354215.jpg
2,/images/apple_pie/1355206.jpg
3,/images/apple_pie/1357950.jpg
4,/images/apple_pie/1361950.jpg
...,...
68170,/images/waffles/981485.jpg
68171,/images/waffles/98238.jpg
68172,/images/waffles/982668.jpg
68173,/images/waffles/995085.jpg


In [23]:
full = noise_generated.merge(clean, how = "left", left_on = "dir", right_on = "dir")

In [24]:
full

,dir,clean_label,noisy_label
0,/images/apple_pie/1005649.jpg,49,3
1,/images/apple_pie/1014775.jpg,3,3
2,/images/apple_pie/1026328.jpg,53,3
3,/images/apple_pie/1028787.jpg,5,3
4,/images/apple_pie/1043283.jpg,33,3
...,...,...,...
75745,/images/waffles/981485.jpg,1,85
75746,/images/waffles/98238.jpg,30,85
75747,/images/waffles/982668.jpg,91,85
75748,/images/waffles/995085.jpg,80,85


In [28]:
full[full.clean_label == full.noisy_label].shape[0]/full.shape[0]

0.2991815181518152

In [29]:
config.r

0.7

In [26]:
len(noise_generated.dir)

75750

In [27]:
noise_generated

,dir,clean_label
0,/images/apple_pie/1005649.jpg,49
1,/images/apple_pie/1014775.jpg,3
2,/images/apple_pie/1026328.jpg,53
3,/images/apple_pie/1028787.jpg,5
4,/images/apple_pie/1043283.jpg,33
...,...,...
75745,/images/waffles/981485.jpg,1
75746,/images/waffles/98238.jpg,30
75747,/images/waffles/982668.jpg,91
75748,/images/waffles/995085.jpg,80
